In [14]:
import cv2
import torch
import os
import numpy as np
import argparse
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.utils as vutils
from network.Transformer import Transformer


#parser = argparse.ArgumentParser()
#parser.add_argument('--input_dir', default = 'test_img')
#parser.add_argument('--load_size', default = 450)
#parser.add_argument('--model_path', default = './pretrained_model')
#parser.add_argument('--style', default = 'Hayao')
#parser.add_argument('--output_dir', default = 'test_output1')
#parser.add_argument('--gpu', type=int, default = -1)

#opt = parser.parse_args()

model_path = 'F:\Work\ML Projects\CartoonGAN-Test-Pytorch-Torch\Shinkai_net_G_float.pth'


Shinkai_net_G_float = torch.load(model_path)
valid_ext = ['.jpg', '.png']

if not os.path.exists("test_output1"): os.mkdir("test_output1")

# load pretrained model
model = Transformer()
model.load_state_dict(Shinkai_net_G_float)
model.eval()

print('CPU mode')
model.float()


# Initialize the video capture device
capture = cv2.VideoCapture(0)

# Initialize a frame counter
frame_count = 0

while True:
    # Capture the current frame
    ret, frame = capture.read()

    # Check if the video capture was successful
    if not ret:
        print("Failed to capture video")
        break

   
    # Save the frame as an image file
    filename = f"frame{frame_count}.jpg"
    cv2.imwrite(filename, frame)
    with torch.no_grad():
    # load image
        input_image = Image.open(filename).convert("RGB")
    
    # resize image, keep aspect ratio
        h = input_image.size[0]
        w = input_image.size[1]
        ratio = h *1.0 / w
        if ratio > 1:
            h = 450
            w = int(h*1.0/ratio)
        else:
            w = 450
            h = int(w * ratio)
        input_image = input_image.resize((h, w), Image.BICUBIC)
        input_image = np.asarray(input_image)
        # RGB -> BGR
        input_image = input_image[:, :, [2, 1, 0]]
        input_image = transforms.ToTensor()(input_image).unsqueeze(0)
        # preprocess, (-1, 1)
        input_image = -1 + 2 * input_image 
        input_image = Variable(input_image, volatile=True).float()
        # forward
        output_image = model(input_image)
        output_image = output_image[0]
        # BGR -> RGB
        output_image = output_image[[2, 1, 0], :, :]
        # deprocess, (0, 1)
        output_image = output_image.data.cpu().float() * 0.5 + 0.5
        # save
        array = output_image.permute(1,2,0).numpy()
        array = (255 * array).astype(np.uint8)

        cv2.imshow('image', array)
        
    
        #output_image_name = os.path.join('test_output1', str(frame_count) + '_' + 'Shinkai' + '.jpg')
        #frame_count = frame_count+1
        #vutils.save_image(output_image,output_image_name )


        # Load the image from disk
        #image = cv2.imread(output_image_name)

        # Convert the image to a Mat object
        #mat = cv2.UMat(image)

        # Display the frame in a window
        #cv2.imshow('image', mat)


    # Wait for a key press to exit
    if cv2.waitKey(1) == 27: # Esc key
        break

# Release the video capture device and close all windows
capture.release()
cv2.destroyAllWindows()





CPU mode


C:\Users\abstu\AppData\Local\Temp\ipykernel_8580\2036234583.py:73: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  input_image = input_image.resize((h, w), Image.BICUBIC)
C:\Users\abstu\AppData\Local\Temp\ipykernel_8580\2036234583.py:80: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  input_image = Variable(input_image, volatile=True).float()
f:\Work\Anaconda\envs\Pytorch\lib\site-packages\torch\nn\functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [36]:
import cv2
import torch
import os
import numpy as np
import argparse
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.utils as vutils
from network.Transformer import Transformer
import time

#parser = argparse.ArgumentParser()
#parser.add_argument('--input_dir', default = 'test_img')
#parser.add_argument('--load_size', default = 450)
#parser.add_argument('--model_path', default = './pretrained_model')
#parser.add_argument('--style', default = 'Hayao')
#parser.add_argument('--output_dir', default = 'test_output1')
#parser.add_argument('--gpu', type=int, default = -1)

#opt = parser.parse_args()

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = 'F:\Work\ML Projects\CartoonGAN-Test-Pytorch-Torch\Hayao_net_G_float.pth'

Shinkai_net_G_float = torch.load(model_path)

# load pretrained model
model = Transformer()
model.load_state_dict(Shinkai_net_G_float)
model.eval()
model.float()
#print('Using', device)

# Initialize the video capture device
capture = cv2.VideoCapture(0)

while True:
    # Capture the current frame
    ret, frame = capture.read()
    

    # Check if the video capture was successful
    if not ret:
        print("Failed to capture video")
        break

    # Save the frame as an image file
    #filename = f"frame{frame_count}.jpg"
    #cv2.imwrite(filename, frame)
    

    with torch.no_grad():

        
        # resize image, keep aspect ratio
        # Resize the frame
        frame = cv2.resize(frame, (450, 338))

        # Convert the frame to a PyTorch tensor
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        frame = transforms.ToTensor()(frame).unsqueeze(0)
        frame = -1 + 2 * frame 
        frame = Variable(frame, volatile=True).float()
        
        # forward
        output_image = model(frame)
        output_image = output_image[0]
        
        # BGR -> RGB
        output_image = output_image[[2, 1, 0], :, :]
        
        # deprocess, (0, 1)
        output_image = output_image.data.cpu().float() * 0.5 + 0.5
        
        # save
        array = output_image.permute(1, 2, 0).numpy()
        array = (255 * array).astype(np.uint8)

        cv2.imshow('image', array)

    # Wait for a key press to exit
    if cv2.waitKey(1) == 27: # Esc key
        break

capture.release()
cv2.destroyAllWindows()


C:\Users\abstu\AppData\Local\Temp\ipykernel_8580\4200631879.py:66: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  frame = Variable(frame, volatile=True).float()
f:\Work\Anaconda\envs\Pytorch\lib\site-packages\torch\nn\functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor([[[[-3.6788e-06, -4.2819e-06, -4.7644e-06,  ..., -9.2272e-06,
           -8.3829e-06, -7.9004e-06],
          [-3.7994e-06, -4.1613e-06, -4.7644e-06,  ..., -8.1417e-06,
           -7.6592e-06, -7.4180e-06],
          [-3.9201e-06, -4.0407e-06, -4.5231e-06,  ..., -7.6592e-06,
           -7.2973e-06, -7.0561e-06],
          ...,
          [ 1.7490e-06,  1.9902e-06,  2.1108e-06,  ..., -1.3087e-05,
           -1.3087e-05, -1.2846e-05],
          [ 1.8696e-06,  1.9902e-06,  1.9902e-06,  ..., -1.3087e-05,
           -1.3087e-05, -1.2846e-05],
          [ 2.2314e-06,  2.3520e-06,  2.3520e-06,  ..., -1.3087e-05,
           -1.3087e-05, -1.2846e-05]],

         [[-5.0056e-06, -5.3675e-06, -5.4881e-06,  ..., -8.2623e-06,
           -8.1417e-06, -7.9004e-06],
          [-5.2469e-06, -5.3675e-06, -5.6087e-06,  ..., -8.1417e-06,
           -8.0210e-06, -8.0210e-06],
          [-5.3675e-06, -5.3675e-06, -5.4881e-06,  ..., -8.1417e-06,
           -8.0210e-06, -8.0210e-06],
          ...,
     

In [9]:

# Release the video capture device and close all windows
capture.release()
cv2.destroyAllWindows()


In [10]:
array.shape

(340, 452, 3)

In [11]:
array = (255 * array).astype(np.uint8)

In [12]:
array.shape

(340, 452, 3)

In [13]:
cv2.imshow('image', array)
cv2.waitKey(0)
cv2.destroyAllWindows()